In [1]:
# First step towards implementing a gui for the library
%gui qt5

In [1]:
import matplotlib as mpl
mpl.use('Qt5Agg')
import sys
import random
import numpy as np

from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import Qt

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT as NavigationToolbar, FigureCanvas
from matplotlib.figure import Figure

class MplCanvas(FigureCanvasQTAgg):

    def __init__(self, parent=None, width=5, height=4, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        super(MplCanvas, self).__init__(fig)
        
        toolbar = NavigationToolbar(self, parent)
        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(toolbar)
        layout.addWidget(self)
        
        self.layout = layout
        
class Slider:
    
    def __init__(self, name, min_val, max_val, ini_val, parent):
        
        name = QtWidgets.QLabel(name, parent=parent)
        name.setAlignment(Qt.AlignCenter | Qt.AlignVCenter)
        
        slider = QtWidgets.QSlider(Qt.Horizontal)
        slider.setRange(min_val, max_val)
        #slider.setFocusPolicy(Qt.NoFocus)      # Removes ugly box
        slider.setValue(ini_val)
    
        label = QtWidgets.QLabel(str(ini_val), parent=parent)
        label.setAlignment(Qt.AlignCenter | Qt.AlignVCenter)
        label.setMinimumWidth(20)
        
        layouth = QtWidgets.QHBoxLayout()
        layouth.addWidget(name)
        layouth.addWidget(slider)
        #layouth.addSpacing(15)
        layouth.addWidget(label)
        
        widget = QtWidgets.QWidget()
        widget.setLayout(layouth)
        
        self.slider = slider
        self.label = label
        self.layouth = layouth
        self.widget = widget
        
    def set_value_changed(self, func):
        
        self.slider.valueChanged.connect(func)

class MainWindow(QtWidgets.QMainWindow):

    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)

        self.canvas = MplCanvas(self, width=5, height=5)
        self.create_sliders()
        self.create_layout()
        
        self.img = np.random.rand(300, 300)
        self.plot_ref = None
        self.plot()

        self.show()
        
    def create_sliders(self):
    
        sl1 = Slider('Lower', 0, 100, 25, self)
        sl1.set_value_changed(self.update_plot)
        sl2 = Slider('Upper', 0, 100, 75, self)
        sl2.set_value_changed(self.update_plot)
        
        self.sl1 = sl1
        self.sl2 = sl2
    
    def create_layout(self):
        
        layoutv = QtWidgets.QVBoxLayout()

        layoutv.addLayout(self.canvas.layout, 1)
        layoutv.addWidget(self.sl1.widget)
        layoutv.addWidget(self.sl2.widget)
        
        widget = QtWidgets.QWidget()
        widget.setLayout(layoutv)
        self.setCentralWidget(widget)

    def plot(self):
        
        self.img = np.random.rand(300, 300)
        threshold1 = 25
        threshold2 = 75
        self.plot_ref = self.canvas.axes.imshow(np.logical_and(self.img>threshold1/100, self.img<threshold2/100), 'gray')
        self.canvas.draw()

    def update_plot(self):
        
        slider1 = self.sl1
        threshold1 = slider1.slider.value()
        slider2 = self.sl2
        threshold2 = slider2.slider.value()
        self.plot_ref.set_data(np.logical_and(self.img>threshold1/100, self.img<threshold2/100))
        
        slider1.label.setText(f"{threshold1}")
        slider2.label.setText(f"{threshold2}")
        
        self.canvas.draw()

app = QtWidgets.QApplication([])
app.aboutToQuit.connect(app.deleteLater)
mw = MainWindow()

TypeError: resizeEvent() takes 1 positional argument but 2 were given

In [ ]:
# Original
import sys
import random
import matplotlib
matplotlib.use('Qt5Agg')

from PyQt5 import QtCore, QtGui, QtWidgets

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


class MplCanvas(FigureCanvasQTAgg):

    def __init__(self, parent=None, width=5, height=4, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        super(MplCanvas, self).__init__(fig)

class MainWindow(QtWidgets.QMainWindow):

    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)

        self.canvas = MplCanvas(self, width=5, height=4, dpi=100)
        self.setCentralWidget(self.canvas)

        n_data = 50
        self.xdata = list(range(n_data))
        self.ydata = [random.randint(0, 10) for i in range(n_data)]

        # We need to store a reference to the plotted line 
        # somewhere, so we can apply the new data to it.
        self._plot_ref = None
        self.update_plot()

        self.show()

        # Setup a timer to trigger the redraw by calling update_plot.
        self.timer = QtCore.QTimer()
        self.timer.setInterval(10)
        self.timer.timeout.connect(self.update_plot)
        self.timer.start()

    def update_plot(self):
        # Drop off the first y element, append a new one.
        self.ydata = self.ydata[1:] + [random.randint(0, 10)]

        # Note: we no longer need to clear the axis.       
        if self._plot_ref is None:
            # First time we have no plot reference, so do a normal plot.
            # .plot returns a list of line <reference>s, as we're
            # only getting one we can take the first element.
            plot_refs = self.canvas.axes.plot(self.xdata, self.ydata, 'r')
            self._plot_ref = plot_refs[0]
        else:
            # We have a reference, we can use it to update the data for that line.
            self._plot_ref.set_ydata(self.ydata)

        # Trigger the canvas to update and redraw.
        self.canvas.draw()

app = QtWidgets.QApplication([])
app.aboutToQuit.connect(app.deleteLater)
mw = MainWindow()